In [11]:
#INSTALL TOOLS
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!pip install -q fastapi uvicorn nest_asyncio pdf2image pytesseract python-multipart

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [12]:
#import & Start fast api App
import nest_asyncio
import uvicorn
from fastapi import FastAPI, UploadFile, File
import threading
import time
app = FastAPI()

@app.get('/')
def home():
  return{"message": "Server is running!"}
  nest_asyncio.apply()


In [13]:
#SIMPLE OCR HELPER

from pdf2image import convert_from_path
import pytesseract
def extract_text_from_pdf(pages):
#def extract_text_from_pdf(pdf_path):
  full=""
  for p in pages:
     full +=pytesseract.image_to_string(p)+"\n"
  return full

In [14]:
# UPLOAD API
@app.post("/upload")
async def upload_file(file: UploadFile= File(...)):
  path= f"tmp/{file.filename}"
  with open(path, "wb") as f:
    f.write(file.file.read())
  text= extract_text(path) #OCR
  return{"text": text[2000]} #return 2000 characters

In [15]:
#Start the server

def run():
  uvicorn.run(app,host="0.0.0.0", port=8000)

  threading.thread(target=run, demon=True).start()
  time.sleep(1)
  print("Server Started on port 8000")

In [16]:
# Public URL using cloudflared
import subprocess
import re
import time

cmd = ["cloudflared", "tunnel", "--url", "http://localhost:8000", "--no-autoupdate"]

# Start the tunnel process
proc = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

public_url = None
start = time.time()

while True:
    line = proc.stdout.readline()
    if not line:
        break

    print(line.strip())

    match = re.search(r"https://.*trycloudflare.com", line)
    if match:
        public_url = match.group(0)
        break

    if time.time() - start > 25:
        break

if public_url:
    print("\nPUBLIC URL:", public_url)
    print("Open docs here:", public_url + "/docs")
else:
    print("Failed to get public URL")


2025-12-03T12:35:16Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-03T12:35:16Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-03T12:35:19Z INF +--------------------------------------------------------------------------------------------+
2025-12-03T12:35:19Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-03T12:35:19Z INF |  https://accommodations-dietary-paxil-microphone.trycl